In [ ]:
#%matplotlib notebook

In [ ]:
#basic imports
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

In [ ]:
# initial conditions
init = {
    "x0": 0,
    "y0": 1,
    "z0": 1.05
}

# parameters
params = {
    "sigma": 10,
    "b": 8/3,
    "r": 50,
    "N": 15000,
    "T": 60
}

In [ ]:
def lorenz_system_with_euler(initial_conditions:dict, paramaters:dict):
    """
    Computes the solutions with explicit euler's method
    :param initial_conditions: Dictionary of initial conditions.
    :param paramaters: Dictionary of parameters.
    :return:
    """

    print(f"Parmaters: {paramaters}\n"
          f"Initial conditions: {initial_conditions}\n")

    # initial conditions and parameters
    x0, y0, z0 = [initial_conditions[key] for key in initial_conditions.keys()]
    sg, b, r, n, tt = [paramaters[key] for key in paramaters.keys()]

    # time sampling
    h = tt / n
    t = np.linspace(0, tt, n, endpoint=False)
    # t = np.arange(0,tt,h)

    # initializing solutions vectors
    x_solutions = np.zeros(t.size)
    y_solutions = np.zeros(t.size)
    z_solutions = np.zeros(t.size)

    # initializing solutions
    x_solutions[0] = x0
    y_solutions[0] = y0
    z_solutions[0] = z0

    # computing
    for i in range(1, t.size):
        x_solutions[i] = x_solutions[i-1] + h * sg * (y_solutions[i-1] - x_solutions[i-1])

        y_solutions[i] = y_solutions[i-1] + h * ( r * x_solutions[i-1] - y_solutions[i-1] - x_solutions[i-1] * z_solutions[i-1] )

        z_solutions[i] = z_solutions[i-1] + h * ( x_solutions[i-1] * y_solutions[i-1] - b * z_solutions[i-1] )

    return t, x_solutions, y_solutions, z_solutions

In [ ]:
t,x,y,z = lorenz_system_with_euler(init,params)

In [ ]:
plt.plot(x,y)
plt.grid(True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x,y,z)

## Stability
Here we will simulate 4 cases.
- First case: when the system is stable. This means that r < 1
- Second case: when the system is in a bifurcation state. This means that r = 1
- Third case: when the system is chaotic. This means that r > 1
- Fourth case: when we get to r = r_critical where the system becomes chaotic.

### First case: stability at r < 1

In [ ]:
# definition of the initial parameters
# initial conditions (not changed)
# parameters changed for r
params["r"] = 0.4

In [ ]:
# resolution and viz
t,x,y,z = lorenz_system_with_euler(init,params)
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x,y,z)

### Second case: r=1

In [ ]:
params["r"] = 1
# resolution and viz
t,x,y,z = lorenz_system_with_euler(init,params)
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x,y,z)

### Third case: r greater than 1

In [ ]:
params["r"] = 3
# resolution and viz
t,x,y,z = lorenz_system_with_euler(init,params)
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x,y,z)

### Fourth case : r_critical

In [ ]:
# let's get the ritical value of r
rh = params["sigma"] * ((params["sigma"] + params["b"] + 3)/(params["sigma"] - params["b"] -1))
print(rh)

In [ ]:
params["r"] = rh
# resolution and viz
t,x,y,z = lorenz_system_with_euler(init,params)
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x,y,z)

## Full animation


In [ ]:
params["r"] = rh + 5
# the rest is the same
t,x,y,z = lorenz_system_with_euler(init,params)

In [ ]:
# Optimized animation: downsample trajectory and limit frames to keep the notebook responsive
max_frames = 2000
# choose evenly spaced indices and ensure uniqueness
frames_idx = np.unique(np.linspace(0, len(x)-1, num=min(len(x), max_frames), dtype=int))
# downsampled arrays used for animation (much smaller than full arrays)
x_ds = x[frames_idx]
y_ds = y[frames_idx]
z_ds = z[frames_idx]

fig = go.Figure()

# Static (downsampled) full trajectory in light gray
fig.add_trace(go.Scatter3d(
    x=x_ds, y=y_ds, z=z_ds,
    mode='lines',
    line=dict(width=2, color='lightgray'),
    name='Trajectoire complète (downsampled)'
))

# Moving particle (single point)
fig.add_trace(go.Scatter3d(
    x=[x_ds[0]], y=[y_ds[0]], z=[z_ds[0]],
    mode='markers',
    marker=dict(size=6, color='red'),
    name='Particule'
))

# Animated partial trajectory (starts empty)
fig.add_trace(go.Scatter3d(
    x=[], y=[], z=[],
    mode='lines',
    line=dict(width=3, color='blue'),
    name='Trajectoire animée'
))

# Build frames that only update the moving point and the partial trajectory using downsampled arrays
frames = []
slider_steps = []
for i in range(len(x_ds)):
    frames.append(go.Frame(
        data=[
            {},  # trace 0 is static (no update)
            go.Scatter3d(x=[x_ds[i]], y=[y_ds[i]], z=[z_ds[i]]),
            go.Scatter3d(x=x_ds[:i+1], y=y_ds[:i+1], z=z_ds[:i+1])
        ],
        name=str(i)
    ))

    slider_steps.append({
        "args": [[str(i)], {"frame": {"duration": 50, "redraw": True}, "mode": "immediate"}],
        "label": str(i),
        "method": "animate"
    })

fig.frames = frames

# Layout with play/pause buttons and a populated slider (uses the prebuilt steps)
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True, "mode": "immediate"}], "label": "▶", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}], "label": "⏸", "method": "animate"}
        ],
        "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False,
        "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"
    }],
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {"font": {"size": 20}, "prefix": "Frame:", "visible": True, "xanchor": "right"},
        "transition": {"duration": 0},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": slider_steps
    }],
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    title="Animation du système de Lorenz (optimisée)"
)

# ensure original updatemenus configuration is preserved
fig.update_layout(updatemenus=[fig.layout.updatemenus[0]])

# fixing the limits of the plots and the aspect ratio, and saving the video in mp4
fig.update_scenes(
    xaxis=dict(range=[-25, 25], autorange=False),
    yaxis=dict(range=[-30, 30], autorange=False),
    zaxis=dict(range=[0, 50], autorange=False),
    aspectmode='data'
)

fig.show()